# PROJEKT MODELU KLASYFIKACYJNEGO PRZEWIDYWANIA STANU MIESZKAŃ NA PODSTAWIE #

## Extra Trees Classifier ##

### maj 2025

In [1]:
import pandas as pd
import mlflow
from pycaret.classification import setup, pull, compare_models, plot_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import resample
import pymysql
from sqlalchemy import create_engine
import numpy as np
from scipy.stats import skewnorm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# tą komórkę uruchom jeżeli bazę danych bierzesz np. z DBeaver, oracle developer itp.

#username = 'root'
#password = '1234'
#host = '127.0.0.1'
#port = 3306  
#database = 'projekt1'
#engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

#df = pd.read_sql("SELECT * FROM saleflats", con=engine)

#engine.dispose()

In [3]:
MLFLOW_EXPERIMENT_NAME = 'Investoro_Stan_Mieszkania'
MLFLOW_TAGS = {'data': 'Investoro_stany', 'library': 'pycaret'}

mlflow.set_tracking_uri("http://localhost:5000")

In [4]:
# tę komórkę uruchom jeśli czerpiesz dane z pliku .csv
df = pd.read_csv('data.csv', sep=',')

In [5]:
df

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
0,27933,NaN,17,Mieszkanie Łódź,W ofercie posiadamy 3500 nowych mieszkań z ryn...,49.50,457150.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Śródmieście",14,65,10.0,8.0,958447.0,NaN,NaN,r5Np-egdKqliywYJ2u1-lr1cwColzIKnjrh_K5zWcDs=
1,28002,NaN,17,Mieszkanie Łódź,W ofercie posiadamy 3500 nowych mieszkań z ryn...,32.50,339350.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Śródmieście",14,65,10.0,8.0,958447.0,NaN,NaN,RBDGAs-5nrFCbsdVCEAltb1cwColzIKnjrh_K5zWcDs=
2,28051,NaN,17,Mieszkanie Łódź,W ofercie posiadamy 3500 nowych mieszkań z ryn...,36.60,366350.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Śródmieście",14,65,10.0,8.0,958447.0,NaN,NaN,Vo0wmr6ebtURRouJ0A8pgb1cwColzIKnjrh_K5zWcDs=
3,28104,NaN,17,Mieszkanie Łódź,W ofercie posiadamy 3500 nowych mieszkań z ryn...,79.00,661450.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Śródmieście",14,65,10.0,8.0,958447.0,NaN,NaN,2iJEGwgVL48PWfJI7iBTWb1cwColzIKnjrh_K5zWcDs=
4,32374,NaN,17,"Nowe mieszkanie Śródmieście, ul. Północna 26","Mieszkanie, znajdujące się w inwestycji Północ...",63.96,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Śródmieście",14,65,10.0,8.0,954700.0,NaN,17421.0,LRhiqL-Lo94oL1ZoMjfEDb1cwColzIKnjrh_K5zWcDs=
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196235,4418401,NaN,2,Mieszkanie na sprzedaż - 59 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,59.99,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14,5,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYftiMRU9ErLfA6XOKdBw9jY=
196236,4418402,NaN,2,Mieszkanie na sprzedaż - 41 m² - 2 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,41.23,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14,5,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYa7D8z7yjun2jGP_aZIReN8=
196237,4418404,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.09,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14,5,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYf9eM73_U8S05NBjfLiez6E=
196238,4418405,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.48,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14,5,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYRXVZdKE7iJxxFPu0Bq0ikc=


In [6]:
# pobranie próbki 10 losowych wierszy
df.sample(10)

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
51384,2075506,NaN,11,"Mieszkanie, Legionowo, Legionowo, 53 m²","Mieszkanie, Legionowo, Legionowo, 53 m² Ogłosz...",53.00,530000.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Legionowski, Legionowo",14,8,1.0,1.0,920806.0,NaN,NaN,YVLFN5oxj_Qq0xMhOXWyhlxslNBbpBprsR-39HjnMC8=
60182,2315368,NaN,19,Mieszkanie na sprzedaż do aranżacji w Ostrołęce,"Adres: Gen. Józefa Hallera 5B, Ostrołęka Powie...",32.00,233600.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Ostrołęka",14,61,NaN,NaN,NaN,NaN,NaN,EHoxsZr1L4RGbvHqhiIdPatLCFuALKtSKAVt1gPg8yU=
42704,1764613,NaN,3,BEZPOŚREDNIO mieszkanie 45m z ogródkiem 65m w ...,"Sprzedam mieszkanie 44,5 metra2. Nieruchomość ...",44.50,784999.0,NaN,NaN,795999.0,...,NaN,"Mazowieckie, Warszawa, Wawer, Anin",14,65,14.0,8.0,918850.0,NaN,NaN,t95nGnPsxF2EdQVJqR4z4AGYSGvKlYoNawdyhAmLdNQ=
40947,1705239,NaN,3,"3 pok, dwustronne, rozkładowe, komunikacja",3 pokoje Praga Południe |3 pokoje do wejścia n...,72.00,979000.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Praga-południe, Grochów",14,65,7.0,8.0,918927.0,NaN,NaN,fAOFd_HgEzstURlNb9Eww1sXq5Wq09EqXqwZKl7vxi8=
2306,89729,NaN,3,PROMO_DAYS_3_Pokoje_z_garderobą_ ! ANTRESOLA !,BIURO SPRZEDAŻY MIESZKAŃ Oferta bezpośrednia. ...,98.80,1343000.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Białołęka Dw...",14,65,3.0,8.0,919358.0,NaN,NaN,Z3o-fsBoyiPzQS2edasGEL1cwColzIKnjrh_K5zWcDs=
48831,2018925,NaN,11,"Mieszkanie, Otwock, Otwock, 34 m²","Mieszkanie, Otwock, Otwock, 34 m² Ogłoszenioda...",34.26,399000.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Otwocki, Otwock",14,17,2.0,1.0,921237.0,NaN,NaN,-mogsJfHfLkdmKVO5SBVe-baRshu3omO9IJePLIRnwA=
137338,3619209,NaN,38,Mieszkanie 3 pokojowe,Oferta aktualna - możliwość umówienia bezpiecz...,53.00,769000.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Mokotów",14,65,5.0,8.0,NaN,NaN,NaN,r3RrOLFt3rJ-BEQIIR-seFI945vZrxCmGc1kx-MElW4=
128623,3466922,NaN,3,4-pokojowe mieszkanie 89m2 + ogródek Bez Prowizji,4-pokojowe mieszkanie numer B2-A004 na parterz...,89.11,1240000.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Targówek, Ul. Myszyniecka",14,65,11.0,8.0,988827.0,NaN,13549.0,C8YO9CX-t6GYuwGOIa1g0-iSS5I-hJJyKu-R1kSO-hE=
23285,1110058,NaN,3,Aluzyjna 39 Gotowe mieszkanie OZE duży balkon,Bezpośrednio Bez PCC2 Bez prowizji Do odbioru ...,54.00,693000.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Nowodwory",14,65,3.0,8.0,919594.0,NaN,NaN,kriPV9_kB0-Hz6e7x9uipiFhR9uKFSRu990qiyDp3Xk=
55474,2179627,NaN,32,"Mieszkanie Warszawa gm. Mokotów Mokotów, Aleja...",Opis oferty Apartament zlokalizowany na zamkni...,93.00,1880000.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Mokotów",14,65,5.0,8.0,NaN,NaN,NaN,u7Ku9hpPjZAH8ZxqZmJt4oaCdEfvBnZExzKfPS9GXW0=


In [7]:
print(df.columns)

Index(['SaleId', 'OriginalId', 'PortalId', 'Title', 'Description', 'Area',
       'Price', 'OfferPrice', 'RealPriceAfterRenovation', 'OriginalPrice',
       'PricePerSquareMeter', 'NumberOfRooms', 'BuiltYear', 'Type',
       'BuildingType', 'BuildingCondition', 'OfferFrom', 'Floor', 'Floors',
       'TypeOfMarket', 'OwnerType', 'DateAddedToDatabase', 'DateAdded',
       'DateLastModification', 'DateLastRaises', 'NewestDate', 'AvailableFrom',
       'Link', 'Phone', 'MainImage', 'OtherImages', 'NumberOfDuplicates',
       'NumberOfRaises', 'NumberOfModifications', 'IsDuplicatePriceLower',
       'IsDuplicatePrivateOwner', 'Score', 'ScorePrecision', 'CommunityScore',
       'NumberOfCommunityComments', 'NumberOfCommunityOpinions', 'Archive',
       'Location', 'VoivodeshipNumber', 'CountyNumber', 'CommunityNumber',
       'KindNumber', 'RegionNumber', 'SubRegionNumber', 'StreetNumber',
       'EncryptedId'],
      dtype='object')


In [8]:
df['BuiltYear'] = pd.to_datetime(df['BuiltYear'], format='%Y', errors='coerce')

In [9]:
df_a = df.dropna(subset=['Description'])

In [10]:
df_a.isnull().sum()

SaleId                            0
OriginalId                   194728
PortalId                          0
Title                             0
Description                       0
Area                            353
Price                         12772
OfferPrice                   179230
RealPriceAfterRenovation     179230
OriginalPrice                174419
PricePerSquareMeter           13316
NumberOfRooms                  2751
BuiltYear                     40253
Type                              0
BuildingType                  42563
BuildingCondition            181505
OfferFrom                      1725
Floor                          9082
Floors                        79747
TypeOfMarket                   4197
OwnerType                     92137
DateAddedToDatabase               0
DateAdded                         0
DateLastModification         149289
DateLastRaises               131955
NewestDate                        0
AvailableFrom                194728
Link                        

In [11]:
df_b = df_a.dropna(subset=['Location'])

In [12]:
df_b.isnull().sum()

SaleId                            0
OriginalId                   194728
PortalId                          0
Title                             0
Description                       0
Area                            353
Price                         12772
OfferPrice                   179230
RealPriceAfterRenovation     179230
OriginalPrice                174419
PricePerSquareMeter           13316
NumberOfRooms                  2751
BuiltYear                     40253
Type                              0
BuildingType                  42563
BuildingCondition            181505
OfferFrom                      1725
Floor                          9082
Floors                        79747
TypeOfMarket                   4197
OwnerType                     92137
DateAddedToDatabase               0
DateAdded                         0
DateLastModification         149289
DateLastRaises               131955
NewestDate                        0
AvailableFrom                194728
Link                        

In [13]:
df_c = df_b.dropna(subset=['BuildingCondition'])

In [14]:
df_c.isnull().sum()

SaleId                           0
OriginalId                   13223
PortalId                         0
Title                            0
Description                      0
Area                            10
Price                         1798
OfferPrice                    5042
RealPriceAfterRenovation      5042
OriginalPrice                12541
PricePerSquareMeter           1827
NumberOfRooms                   43
BuiltYear                     1973
Type                             0
BuildingType                  3187
BuildingCondition                0
OfferFrom                        0
Floor                          360
Floors                        1958
TypeOfMarket                   129
OwnerType                     4812
DateAddedToDatabase              0
DateAdded                        0
DateLastModification         11674
DateLastRaises               11322
NewestDate                       0
AvailableFrom                13223
Link                             0
Phone               

In [15]:
df_c

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
178881,4236909,NaN,10,Mieszkanie na sprzedaż - 56 m² - 3 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,56.52,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14,65,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF1I945vZrxCmGc1kx-MElW4=
178882,4236910,NaN,10,Mieszkanie na sprzedaż - 39 m² - 3 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,39.44,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14,65,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF2b3MhONsWHx4CM5o0osslE=
178884,4236917,NaN,10,Mieszkanie na sprzedaż - 57 m² - 4 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,57.72,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14,65,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF5P7_4V7ShzJmmF8PHTVfng=
178886,4236925,NaN,10,Mieszkanie na sprzedaż - 34 m² - 2 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,34.69,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14,65,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF-baRshu3omO9IJePLIRnwA=
178887,4236928,NaN,10,Mieszkanie na sprzedaż - 37 m² - 2 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,37.03,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14,65,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF4_bCM8lJ876moazm9h2OAo=
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196235,4418401,NaN,2,Mieszkanie na sprzedaż - 59 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,59.99,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14,5,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYftiMRU9ErLfA6XOKdBw9jY=
196236,4418402,NaN,2,Mieszkanie na sprzedaż - 41 m² - 2 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,41.23,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14,5,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYa7D8z7yjun2jGP_aZIReN8=
196237,4418404,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.09,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14,5,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYf9eM73_U8S05NBjfLiez6E=
196238,4418405,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.48,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14,5,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYRXVZdKE7iJxxFPu0Bq0ikc=


In [16]:
df_c['Description'] = df_c['Description'].str.slice(0, 3000)

In [17]:
# Przygotuj wektorizer z limitem
vectorizer = CountVectorizer(max_features=1500)  # np. 1500 najczęstszych słów
X_bow = vectorizer.fit_transform(df_c["Description"])

# Konwertuj do DataFrame
df_bow = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Dołącz do oryginalnych danych (z wyjątkiem kolumny Description)
df_c = df_c.drop(columns=['Description']).reset_index(drop=True)
df_bow = df_bow.reset_index(drop=True)
df_prepared = pd.concat([df_c, df_bow], axis=1)

In [18]:
import joblib
import os

output_dir = "DATA_STAN_OUT"
os.makedirs(output_dir, exist_ok=True)

vectorizer_path = os.path.join(output_dir, "count_vectorizer.joblib")
joblib.dump(vectorizer, vectorizer_path)
print(f"CountVectorizer zapisany w: {vectorizer_path}")

CountVectorizer zapisany w: DATA_STAN_OUT\count_vectorizer.joblib


In [19]:
df_prepared.info

<bound method DataFrame.info of         SaleId  OriginalId  PortalId  \
0      4236909         NaN        10   
1      4236910         NaN        10   
2      4236917         NaN        10   
3      4236925         NaN        10   
4      4236928         NaN        10   
...        ...         ...       ...   
13218  4418401         NaN         2   
13219  4418402         NaN         2   
13220  4418404         NaN         2   
13221  4418405         NaN         2   
13222  4418406         NaN         2   

                                                   Title   Area  Price  \
0      Mieszkanie na sprzedaż - 56 m² - 3 pokoje Wars...  56.52    NaN   
1      Mieszkanie na sprzedaż - 39 m² - 3 pokoje Wars...  39.44    NaN   
2      Mieszkanie na sprzedaż - 57 m² - 4 pokoje Wars...  57.72    NaN   
3      Mieszkanie na sprzedaż - 34 m² - 2 pokoje Wars...  34.69    NaN   
4      Mieszkanie na sprzedaż - 37 m² - 2 pokoje Wars...  37.03    NaN   
...                                        

In [20]:
df_prepared[df_prepared.duplicated()]

,SaleId,OriginalId,PortalId,Title,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,PricePerSquareMeter,...,świetną,świeżo,świeżym,żabka,że,życia,życie,życiem,żłobek,żłobki


In [21]:
df_prepared.nunique()

SaleId        13223
OriginalId        0
PortalId         17
Title          9818
Area           3826
              ...  
życia             7
życie             4
życiem            3
żłobek            4
żłobki            4
Length: 1550, dtype: int64

In [22]:
df_prepared.isnull()

,SaleId,OriginalId,PortalId,Title,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,PricePerSquareMeter,...,świetną,świeżo,świeżym,żabka,że,życia,życie,życiem,żłobek,żłobki
0,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13218,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
13219,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
13220,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
13221,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [23]:
df_prepared.isnull().sum()

SaleId            0
OriginalId    13223
PortalId          0
Title             0
Area             10
              ...  
życia             0
życie             0
życiem            0
żłobek            0
żłobki            0
Length: 1550, dtype: int64

In [24]:
Q1 = df_prepared["Price"].quantile(0.25)
Q3 = df_prepared["Price"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_prep_p = df_prepared[~((df_prepared["Price"] < lower_bound) | (df_prepared["Price"] > upper_bound))]

In [25]:
Q1 = df_prep_p["PricePerSquareMeter"].quantile(0.25)
Q3 = df_prep_p["PricePerSquareMeter"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_prep_a = df_prep_p[~((df_prep_p["PricePerSquareMeter"] < lower_bound) | (df_prep_p["PricePerSquareMeter"] > upper_bound))]

In [26]:
Q1 = df_prep_a["Area"].quantile(0.25)
Q3 = df_prep_a["Area"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_prepared2 = df_prep_a[~((df_prep_a["Area"] < lower_bound) | (df_prep_a["Area"] > upper_bound))]

In [27]:
df_prepared2["BuildingCondition"].value_counts(normalize=True)

BuildingCondition
DEVELOPER_STATE     0.422333
AFTER_RENOVATION    0.393079
FOR_RENOVATION      0.099931
GOOD                0.084657
Name: proportion, dtype: float64

In [28]:
df_prepared2.nunique()

SaleId        11588
OriginalId        0
PortalId         17
Title          8577
Area           3234
              ...  
życia             7
życie             4
życiem            3
żłobek            4
żłobki            4
Length: 1550, dtype: int64

In [29]:
unique_btype=df_prepared2['BuildingType'].unique()

unique_btype

array([nan, 'Blok', 'Apartamentowiec', 'Pozostałe', 'Kamienica'],
      dtype=object)

In [30]:
print(df_prepared2['BuildingCondition'].nunique())

4


In [31]:
df_prepared2.isnull().sum()

SaleId            0
OriginalId    11588
PortalId          0
Title             0
Area             10
              ...  
życia             0
życie             0
życiem            0
żłobek            0
żłobki            0
Length: 1550, dtype: int64

In [32]:
df_prepared3=df_prepared2.dropna(subset=['BuildingCondition'])

In [33]:
df_prepared3.loc[df_prepared3['TypeOfMarket'] == 'pierwotny', 'BuildingCondition'] = 'DEVELOPER_STATE'

In [34]:
df_prepared3['BuiltYear'] = pd.to_datetime(df_prepared3['BuiltYear'], format='%Y', errors='coerce')

In [35]:
df_prepared3.loc[df_prepared3['BuiltYear'].dt.year >= 2024, 'BuildingCondition'] = 'DEVELOPER_STATE'

In [36]:
df_prepared3 = df_prepared3[~((df_prepared3['Link'].str.contains('otodom', case=False, na=False)) & 
                              (df_prepared3['BuildingCondition'] == 'AFTER_RENOVATION'))]

In [37]:
unique_ctype=df_prepared3['BuildingCondition'].unique()

unique_ctype

array(['DEVELOPER_STATE', 'AFTER_RENOVATION', 'FOR_RENOVATION', 'GOOD'],
      dtype=object)

In [38]:
df_prepared3["BuildingCondition"].value_counts(normalize=True)

BuildingCondition
DEVELOPER_STATE     0.569412
AFTER_RENOVATION    0.205624
FOR_RENOVATION      0.128598
GOOD                0.096365
Name: proportion, dtype: float64

In [39]:
print("Rozpoczynam ręczne balansowanie klas przed setup...")
df_to_balance = df_prepared3.copy() # Użyj df_prepared3 po wszystkich czyszczeniach

# Sprawdź ile rekordów ma każda klasa
class_counts = df_to_balance['BuildingCondition'].value_counts()
print("Liczebność klas przed zbalansowaniem:")
print(class_counts)

# Znajdź liczebność klasy większościowej, jeśli chcemy robić oversampling do niej,
# lub mniejszościowej, jeśli chcemy robić undersampling. SMOTE robi oversampling.
# Dla SMOTE z imblearn, nie musimy ręcznie robić resample, tylko przekazać dane do SMOTE.

from imblearn.over_sampling import SMOTE

# Przygotuj dane dla SMOTE
X_to_balance = df_to_balance.drop(columns=['BuildingCondition'])
y_to_balance = df_to_balance['BuildingCondition']

# Sprawdzenie, czy wszystkie kolumny w X_to_balance są numeryczne lub mogą być przekonwertowane.
# SMOTE nie działa dobrze z danymi kategorycznymi w formie stringów.
# W Twoim df_prepared3 masz kolumny liczbowe (bag-of-words) i kilka innych.
# Te inne (np. Location, BuildingType) muszą być odpowiednio zakodowane PRZED SMOTE.
# PyCaret robi to automatycznie, ale jeśli robimy SMOTE ręcznie, musimy to zrobić sami.
# To komplikuje sprawę.

# Alternatywnie, dla prostoty, możemy spróbować prostego resamplingu (jak było w Twoim kodzie)
# do liczby najmniejszej klasy (undersampling) lub do największej (oversampling z replace=True).
# SMOTE jest lepszy, ale wymaga bardziej ostrożnego przygotowania danych.

# Wróćmy do oryginalnego pomysłu z resample dla prostoty, jeśli SMOTE ręczny jest zbyt skomplikowany:
min_count = class_counts.min() # dla undersamplingu
# lub:
# target_count = class_counts.max() # dla oversamplingu
# lub ustal konkretną liczbę, np. średnią, medianę.

print(f"Balansowanie klas przez undersampling do: {min_count} próbek na klasę.")
dfs_balanced_list = []
for condition_class in class_counts.index:
    df_condition = df_to_balance[df_to_balance['BuildingCondition'] == condition_class]
    df_condition_resampled = resample(df_condition,
                                    replace=False, # False dla undersamplingu
                                    n_samples=min_count, # lub target_count z replace=True dla oversamplingu
                                    random_state=42)
    dfs_balanced_list.append(df_condition_resampled)

df_balanced_for_setup = pd.concat(dfs_balanced_list).reset_index(drop=True)
print("\nLiczebność klas PO zbalansowaniu:")
print(df_balanced_for_setup['BuildingCondition'].value_counts())
print(f"Kształt df_balanced_for_setup: {df_balanced_for_setup.shape}")

Rozpoczynam ręczne balansowanie klas przed setup...
Liczebność klas przed zbalansowaniem:
BuildingCondition
DEVELOPER_STATE     5123
AFTER_RENOVATION    1850
FOR_RENOVATION      1157
GOOD                 867
Name: count, dtype: int64
Balansowanie klas przez undersampling do: 867 próbek na klasę.

Liczebność klas PO zbalansowaniu:
BuildingCondition
DEVELOPER_STATE     867
AFTER_RENOVATION    867
FOR_RENOVATION      867
GOOD                867
Name: count, dtype: int64
Kształt df_balanced_for_setup: (3468, 1550)


In [40]:
# df_balanced["BuildingCondition"].value_counts(normalize=True)

In [41]:
df_balanced_for_setup

,SaleId,OriginalId,PortalId,Title,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,PricePerSquareMeter,...,świetną,świeżo,świeżym,żabka,że,życia,życie,życiem,żłobek,żłobki
0,4252181,NaN,3,1-pokojowe mieszkanie 31m2 + ogródek,31.11,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,2,0,0,1,0
1,4303399,NaN,11,"Mieszkanie, Warszawa, Wawer, 58 m²",57.64,755084.0,801327.0,1216333.33,NaN,13100.00,...,0,0,0,0,0,0,0,0,0,0
2,4274912,NaN,3,Nowe mieszkanie z ogródkiem przy stacji Pkp,25.00,450000.0,NaN,NaN,NaN,18000.00,...,0,0,0,0,0,0,0,0,0,0
3,4281470,NaN,3,"LOFT, TARAS 45 m2, SAUNA , METRO, CICHE",93.50,1482000.0,1650000.0,2257145.67,NaN,15850.27,...,0,0,0,0,0,0,0,0,0,0
4,4291264,NaN,3,"Nowe, komfortowe mieszkanie 47 m2 z klimatyzacją",47.00,995500.0,789000.0,999166.67,982500.0,20904.26,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3463,4264068,NaN,11,"Mieszkanie, Warszawa, Mokotów, 48 m²",48.40,870000.0,1049000.0,1178937.33,NaN,17975.21,...,0,0,0,1,1,0,0,0,0,0
3464,4295764,NaN,17,"Mieszkanie, ul. Krępowieckiego",42.56,660000.0,709990.0,830866.67,NaN,15507.52,...,0,0,0,0,1,0,0,0,0,0
3465,4418034,NaN,11,"Mieszkanie, Radom, Śródmieście, 45 m²",44.85,349000.0,340000.0,394666.67,NaN,7781.49,...,0,0,0,0,0,0,0,0,0,0
3466,4339525,NaN,3,Słoneczne z dwoma balkonami i garażem,54.85,820000.0,922500.0,966666.67,NaN,14949.86,...,0,0,0,0,0,0,0,0,0,0


In [42]:
train_df_for_setup = df_balanced_for_setup.sample(frac=0.9, random_state=42)
external_holdout_df = df_balanced_for_setup.drop(train_df_for_setup.index) # Zmieńmy nazwę dla jasności
print(f"Kształt train_df_for_setup: {train_df_for_setup.shape}")
print(f"Kształt external_holdout_df: {external_holdout_df.shape}")

Kształt train_df_for_setup: (3121, 1550)
Kształt external_holdout_df: (347, 1550)


In [43]:
df_balanced_for_setup['VoivodeshipNumber'] = df_balanced_for_setup['VoivodeshipNumber'].astype(str)
df_balanced_for_setup['CountyNumber'] = df_balanced_for_setup['CountyNumber'].astype(str)
df_balanced_for_setup['CommunityNumber'] = df_balanced_for_setup['CommunityNumber'].astype(str)
df_balanced_for_setup['KindNumber'] = df_balanced_for_setup['KindNumber'].astype(str)
df_balanced_for_setup['RegionNumber'] = df_balanced_for_setup['RegionNumber'].astype(str)
if 'StreetNumber' in df_prepared.columns:
    df_balanced_for_setup['StreetNumber'] = df_balanced_for_setup['StreetNumber'].astype(str)

In [44]:
print(f"Kształt df_prepared3 PRZED resetem indeksu: {df_balanced_for_setup.shape}")
print(f"Pierwsze 5 indeksów df_prepared3 PRZED resetem: {df_balanced_for_setup.index[:5].tolist()}") # .tolist() dla lepszego wyświetlania
if not df_balanced_for_setup.index.is_unique:
    print("UWAGA: Indeks df_prepared3 nie jest unikalny przed resetem!")
    # Jeśli indeks nie jest unikalny, może to być dodatkowy problem.
    # Rozważ df_prepared3 = df_prepared3.loc[~df_prepared3.index.duplicated(keep='first')] przed resetem.
    # Ale reset_index(drop=True) powinien i tak stworzyć nowy, unikalny indeks.

df_balanced_for_setup = df_balanced_for_setup.reset_index(drop=True)

print(f"Kształt df_prepared3 PO resecie indeksu: {df_balanced_for_setup.shape}")
print(f"Pierwsze 5 indeksów df_prepared3 PO resecie: {df_balanced_for_setup.index[:5].tolist()}")

Kształt df_prepared3 PRZED resetem indeksu: (3468, 1550)
Pierwsze 5 indeksów df_prepared3 PRZED resetem: [0, 1, 2, 3, 4]
Kształt df_prepared3 PO resecie indeksu: (3468, 1550)
Pierwsze 5 indeksów df_prepared3 PO resecie: [0, 1, 2, 3, 4]


In [45]:
# Reset indeksu dla danych przekazywanych do setup
print(f"Resetowanie indeksu dla train_df_for_setup...")
train_df_for_setup = train_df_for_setup.reset_index(drop=True)
# external_holdout_df też powinien mieć zresetowany indeks dla spójności
external_holdout_df = external_holdout_df.reset_index(drop=True)

Resetowanie indeksu dla train_df_for_setup...


In [46]:
categorical_features_initial = [
    'BuildingType', 
    'TypeOfMarket', 'OwnerType', 'Type', 'OfferFrom',
    'VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber',
    'RegionNumber', 'Location'
]
numeric_features_initial = [
    'Area', 'NumberOfRooms', 'Floor', 'Floors', 'CommunityScore', 'Price' 
]
date_features_initial = ['BuiltYear']

# Filtruj cechy, które faktycznie istnieją w df_prepared3
categorical_features_to_use = [col for col in categorical_features_initial if col in train_df_for_setup.columns]
numeric_features_to_use = [col for col in numeric_features_initial if col in train_df_for_setup.columns]
date_features_to_use = [col for col in date_features_initial if col in train_df_for_setup.columns]

# Definicja cech ignorowanych
ignore_features_list = [
    'SaleId', 'OriginalId', 'PortalId', 'Title', 
    'OfferPrice', 'RealPriceAfterRenovation', 'OriginalPrice',
    'PricePerSquareMeter', 'DateAddedToDatabase', 'DateAdded',
    'DateLastModification', 'DateLastRaises', 'NewestDate',
    'AvailableFrom', 'Link', 'Phone', 'MainImage', 'OtherImages',
    'NumberOfDuplicates', 'NumberOfRaises', 'NumberOfModifications',
    'IsDuplicatePriceLower', 'IsDuplicatePrivateOwner', 'Score', 'ScorePrecision',
    'NumberOfCommunityComments', 'NumberOfCommunityOpinions', 'Archive',
    'SubRegionNumber', 'EncryptedId','id', 'link',
    'StreetNumber'
]
# Filtruj ignorowane cechy, aby upewnić się, że są w DataFrame
ignore_features_list = [col for col in ignore_features_list if col in df_to_balance.columns]

transformed_exp = setup(
    data=train_df_for_setup,
    target='BuildingCondition',
    verbose=True,
    session_id=123,
    log_experiment=True,
    experiment_name=MLFLOW_EXPERIMENT_NAME,
    log_data=True,
    log_plots=True,
    fix_imbalance=False,
    categorical_features=categorical_features_to_use,
    numeric_features=numeric_features_to_use,
    date_features=date_features_to_use,
    # text_features=['opis_m'],
    ignore_features=ignore_features_list,
    ordinal_features={'BuildingType': ['Pozostałe', 'Blok', 'Apartamentowiec', 'Kamienica'],
                     },
)
best_transformed_model = transformed_exp.compare_models()
transformed_metrics_df = pull()

,Description,Value
0,Session id,123
1,Target,BuildingCondition
2,Target type,Multiclass
3,Target mapping,"AFTER_RENOVATION: 0, DEVELOPER_STATE: 1, FOR_RENOVATION: 2, GOOD: 3"
4,Original data shape,"(3121, 1550)"
5,Transformed data shape,"(3121, 1560)"
6,Transformed train set shape,"(2184, 1560)"
7,Transformed test set shape,"(937, 1560)"
8,Ignore features,31
9,Ordinal features,1


AttributeError: 'ThreadLocalVariable' object has no attribute 'copy'

In [ ]:
import json
import os

# 'train_df_for_setup' to DataFrame przekazany do setup()
# To jest schemat, który musimy odtworzyć w skrypcie predykcyjnym
expected_columns = list(train_df_for_setup.columns)

# Usuwamy target, bo nie będzie go w danych do predykcji
if 'BuildingCondition' in expected_columns:
    expected_columns.remove('BuildingCondition')

output_dir = "DATA_STAN_OUT"
os.makedirs(output_dir, exist_ok=True)

columns_path = os.path.join(output_dir, "expected_columns.json")
with open(columns_path, 'w') as f:
    json.dump(expected_columns, f)
print(f"Lista oczekiwanych kolumn ({len(expected_columns)} szt.) zapisana w: {columns_path}")

In [ ]:
lightgbm_model = transformed_exp.create_model('lightgbm')

In [ ]:
transformed_exp.dataset.head()

In [ ]:
print("Rozpoczynam strojenie LightGBM...")
tuned_lightgbm = transformed_exp.tune_model(lightgbm_model, optimize='F1', n_iter=10, search_library='scikit-optimize', search_algorithm='bayesian')

In [ ]:
model_do_predykcji = lightgbm_model

In [ ]:
try:
    print("Próbuję sfinalizować model_do_predykcji...")
    final_lightgbm_model = transformed_exp.finalize_model(model_do_predykcji)
    print("Model sfinalizowany pomyślnie.")
    model_do_uzycia_w_skrypcie_predykcji = final_lightgbm_model
except IndexError as e:
    print(f"BŁĄD podczas finalize_model: {e}")
    print("Używam modelu przed finalizacją do dalszych predykcji.")
    model_do_uzycia_w_skrypcie_predykcji = model_do_predykcji # Wracamy do modelu przed finalizacją
except Exception as e:
    print(f"Inny BŁĄD podczas finalize_model: {e}")
    print("Używam modelu przed finalizacją do dalszych predykcji.")
    model_do_uzycia_w_skrypcie_predykcji = model_do_predykcji

In [ ]:
from sklearn.pipeline import Pipeline
import joblib
import os

output_dir = "DATA_STAN_OUT"
os.makedirs(output_dir, exist_ok=True)

# `final_lightgbm_model` to jest pełny potok PyCaret
# Tworzymy nowy potok, który zawiera wszystkie kroki OPRÓCZ ostatniego (modelu)
transformer_steps = final_lightgbm_model.steps[:-1]
transformer_pipeline = Pipeline(steps=transformer_steps)

# Zapisujemy ten potok transformerów
transformers_path = os.path.join(output_dir, "transformers_pipeline.pkl")
joblib.dump(transformer_pipeline, transformers_path)
print(f"Potok transformerów zapisany w: {transformers_path}")

# Wyciągamy i zapisujemy czysty model LGBM
clean_lgbm_model = final_lightgbm_model.steps[-1][1]
clean_model_path = os.path.join(output_dir, "clean_lgbm_model.pkl")
joblib.dump(clean_lgbm_model, clean_model_path)
print(f"Czysty model LGBM zapisany w: {clean_model_path}")

In [ ]:
if 'model_do_uzycia_w_skrypcie_predykcji' in locals():
    transformed_exp.save_model(model_do_uzycia_w_skrypcie_predykcji, "model_LGBM_stan")
    print("Model LGBM zapisany jako model_LGBM_stan.pkl")
else:
    print("Nie zdefiniowano modelu do zapisu. Sprawdź poprzednie kroki.")

In [ ]:
from pycaret.classification import get_config
X = get_config("X")
print(X.shape)

In [ ]:
y = get_config("y")
print(y.shape)

In [ ]:
# W Twoim przepływie, model który chcesz ocenić na holdout_df i którego metryki chcesz zalogować
# to ten, który został sfinalizowany i zapisany.
if 'model_do_uzycia_w_skrypcie_predykcji' in locals() and model_do_uzycia_w_skrypcie_predykcji is not None:
    model_do_oceny_na_holdout = model_do_uzycia_w_skrypcie_predykcji
    print(f"Używam 'model_do_uzycia_w_skrypcie_predykcji' (sfinalizowany {type(model_do_uzycia_w_skrypcie_predykcji.steps[-1][1]).__name__}) do oceny na holdout.")
elif 'final_lightgbm_model' in locals() and final_lightgbm_model is not None: # Alternatywna nazwa, jeśli tej użyłeś
    model_do_oceny_na_holdout = final_lightgbm_model
    print(f"Używam 'final_lightgbm_model' (sfinalizowany {type(final_lightgbm_model.steps[-1][1]).__name__}) do oceny na holdout.")
else:
    # Jako fallback, jeśli powyższe zawiodą, spróbujmy użyć modelu przed finalizacją
    if 'model_do_predykcji' in locals() and model_do_predykcji is not None:
         model_do_oceny_na_holdout = model_do_predykcji
         print(f"OSTRZEŻENIE: Sfinalizowany model nie znaleziony, używam 'model_do_predykcji' ({type(model_do_predykcji).__name__}) przed finalizacją.")
    elif 'lightgbm_model' in locals() and lightgbm_model is not None:
        model_do_oceny_na_holdout = lightgbm_model
        print(f"OSTRZEŻENIE: Sfinalizowany model nie znaleziony, używam 'lightgbm_model' ({type(lightgbm_model).__name__}) przed finalizacją.")
    else:
        raise NameError("Nie zdefiniowano odpowiedniego modelu (np. model_do_uzycia_w_skrypcie_predykcji, final_lightgbm_model, model_do_predykcji) do oceny na zbiorze holdout.")

# holdout_df został stworzony z df_balanced_for_setup w komórce 41.
# df_balanced_for_setup był użyty w setup().
# predict_model oczekuje danych w formie "surowej", takiej jakiej przekazano do setup().
# Więc holdout_df jest poprawnym wejściem tutaj.
print(f"Wykonywanie predykcji na zbiorze holdout_df ({external_holdout_df.shape}) używając modelu: {type(model_do_oceny_na_holdout.steps[-1][1]).__name__ if hasattr(model_do_oceny_na_holdout, 'steps') else type(model_do_oceny_na_holdout).__name__}")
predictions_on_holdout = transformed_exp.predict_model(estimator=model_do_oceny_na_holdout, data=external_holdout_df)

# Pobierz ramkę danych z metrykami dla tej predykcji
predict_holdout_df = transformed_exp.pull() # Ta linia tworzy predict_holdout_df
print("Metryki z predykcji na holdout_df:")
display(predict_holdout_df)

# Logowanie do MLflow
mlflow_exp = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
if mlflow_exp is None:
    print(f"BŁĄD: Eksperyment MLflow '{MLFLOW_EXPERIMENT_NAME}' nie istnieje. Nie można zalogować metryk.")
else:
    runs_df = mlflow.search_runs(experiment_ids=[mlflow_exp.experiment_id], order_by=["start_time DESC"])

    if not runs_df.empty:
        # Znajdź ostatni główny przebieg (nie ten od `predict_model`)
        # Przebiegi `predict_model` są często zagnieżdżone i krótkie.
        # Chcemy zalogować do przebiegu, w którym trenowaliśmy model.
        # Zakładamy, że główny przebieg jest tym stworzonym przez `setup` lub `create_model`.
        # Możemy próbować znaleźć przebieg, który NIE ma tagu `mlflow.source.name` wskazującego na `pycaret.internal.pipeline`
        # lub po prostu wziąć ostatni, jeśli struktura jest prosta.
        
        # Logika znajdowania odpowiedniego run_id może wymagać dostosowania.
        # Dla prostoty, weźmiemy ostatni run, ale to może nie być zawsze główny run modelu.
        # Idealnie, `run_id` powinno być pobrane z obiektu `transformed_exp` po trenowaniu modelu.
        # PyCaret automatycznie loguje do aktywnego przebiegu.
        # Jeśli chcemy dodać do przebiegu stworzonego przez `setup()` lub `create_model('lightgbm')`,
        # musielibyśmy przechwycić jego run_id wcześniej.
        
        # Spróbujmy zalogować jako nowy zagnieżdżony przebieg do ostatniego głównego przebiegu
        # lub jeśli to trudne, po prostu zalogujmy do ostatniego przebiegu (co może być przebiegiem `predict_model`)
        run_id_to_log_into = runs_df['run_id'].iloc[0] # Bierzemy najnowszy run
        
        if not predict_holdout_df.empty and "Model" in predict_holdout_df.columns:
            metrics_dict = predict_holdout_df.drop(columns=["Model"]).iloc[0].to_dict()

            with mlflow.start_run(run_id=run_id_to_log_into, nested=True) as nested_run_holdout_metrics:
                print(f"Logowanie metryk holdout do MLflow run_id: {run_id_to_log_into} (nested run: {nested_run_holdout_metrics.info.run_id})")
                for key, value in metrics_dict.items():
                    if isinstance(value, (int, float, np.number)):
                        mlflow.log_metric(f"holdout_{key}", value)
                    else:
                        print(f"Pominięto logowanie metryki 'holdout_{key}' o wartości '{value}' (nie jest numeryczna).")
                mlflow.set_tag("metrics_source", "holdout_df_evaluation")
                mlflow.set_tag("model_evaluated_on_holdout", f"{type(model_do_oceny_na_holdout.steps[-1][1]).__name__ if hasattr(model_do_oceny_na_holdout, 'steps') else type(model_do_oceny_na_holdout).__name__}")

        else:
            print("BŁĄD: Ramka danych predict_holdout_df jest pusta lub nie zawiera kolumny 'Model'. Nie można zalogować metryk.")
    else:
        print(f"BŁĄD: Nie znaleziono żadnych przebiegów dla eksperymentu '{MLFLOW_EXPERIMENT_NAME}'. Nie można zalogować metryk.")

In [ ]:
mlflow_exp = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
runs_df = mlflow.search_runs([mlflow_exp.experiment_id], order_by=["start_time DESC"])
run_id = runs_df['run_id'].values[0]
metrics = predict_holdout_df.drop(columns=["Model"]).to_dict(orient='records')[0]

with mlflow.start_run(run_id=run_id, nested=True):
    for key, value in metrics.items():
        mlflow.log_metric(key, value)

In [ ]:
#predictions = transformed_exp.predict_model(best_final_model)
holdout_predictions = transformed_exp.predict_model(lightgbm_model) # Zapisz wynik predykcji
display(holdout_predictions.head()) # Wyświetl kilka pierwszych predykcji

In [ ]:
#predicted_column = predict_holdout_df['prediction_label']
#print(predicted_column)
holdout_metrics_df = transformed_exp.pull() # Teraz pobierz metryki z tej predykcji
display(holdout_metrics_df)
predicted_column = holdout_predictions['prediction_label'] # Jeśli chcesz samą kolumnę predykcji
print(predicted_column)

In [ ]:
df_last=holdout_predictions[['prediction_label','BuildingCondition', ]]

In [ ]:
df_last.to_csv('data_full.csv')

In [ ]:
print("Rozpoczynam dodawanie kolumny predykcji do całego zbioru data.csv...")

# 1. Wczytaj oryginalne dane
df_original_all_data = pd.read_csv('data.csv', sep=',')
print(f"Wczytano oryginalne dane: {df_original_all_data.shape[0]} wierszy.")

In [ ]:
# 2. Utwórz kopię roboczą DataFrame do preprocessingu dla całego zbioru
df_to_preprocess_all = df_original_all_data.copy()

In [ ]:
# 3. Preprocessing całego zbioru danych (musi odzwierciedlać kroki prowadzące do df_prepared3,
#    który był wejściem do setup())

# Konwersja BuiltYear na datetime (jak w komórce 8)
df_to_preprocess_all['BuiltYear'] = pd.to_datetime(df_to_preprocess_all['BuiltYear'], format='%Y', errors='coerce')

# Obsługa NaN w Description i Location przed CountVectorizer
df_to_preprocess_all['Description'] = df_to_preprocess_all['Description'].fillna('')
df_to_preprocess_all['Location'] = df_to_preprocess_all['Location'].fillna('') # Jeśli Location było używane

# Przycięcie długości Description (jak w komórce 16)
df_to_preprocess_all['Description'] = df_to_preprocess_all['Description'].str.slice(0, 3000)

# Zastosowanie nauczonego CountVectorizer (z komórki 17)
print("Transformuję kolumnę 'Description' dla całego zbioru za pomocą CountVectorizer...")
X_bow_for_all_pred = vectorizer.transform(df_to_preprocess_all["Description"])
df_bow_for_all_pred = pd.DataFrame(X_bow_for_all_pred.toarray(), columns=vectorizer.get_feature_names_out(), index=df_to_preprocess_all.index)

# Połącz cechy Bag-of-Words z resztą DataFrame
df_processed_for_all_prediction = df_to_preprocess_all.drop(columns=['Description']) # Usuń oryginalną kolumnę tekstową
df_processed_for_all_prediction = pd.concat([df_processed_for_all_prediction, df_bow_for_all_pred], axis=1)
print(f"Dane po dodaniu cech BoW: {df_processed_for_all_prediction.shape[1]} kolumn.")

# Konwersja typów kolumn numerycznych na string (jak w komórce 42)
string_conversion_cols = ['VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber', 'RegionNumber']
if 'StreetNumber' in df_processed_for_all_prediction.columns:
    string_conversion_cols.append('StreetNumber')

print(f"Konwertuję kolumny na string: {string_conversion_cols}")
for col in string_conversion_cols:
    if col in df_processed_for_all_prediction.columns:
        df_processed_for_all_prediction[col] = df_processed_for_all_prediction[col].astype(str)

In [ ]:
print("Rozpoczynam predykcję dla WYPEŁNIENIA brakujących wartości w 'BuildingCondition' w data.csv...")

# 1. Wczytaj oryginalne dane
df_oryginal_do_wypelnienia_nan = pd.read_csv('data.csv', sep=',')
print(f"Wczytano oryginalne dane: {df_oryginal_do_wypelnienia_nan.shape[0]} wierszy.")
print(f"Liczba NaN w 'BuildingCondition' przed wypełnieniem: {df_oryginal_do_wypelnienia_nan['BuildingCondition'].isna().sum()}")

# 2. Utwórz kopię roboczą DataFrame do preprocessingu
df_do_preprocessingu_dla_nan = df_oryginal_do_wypelnienia_nan.copy()

# 3. Preprocessing danych (musi odzwierciedlać kroki prowadzące do danych,
#    na których trenowany był model użyty w `setup()`, np. df_balanced_for_setup lub df_prepared3)

# Konwersja BuiltYear na datetime
df_do_preprocessingu_dla_nan['BuiltYear'] = pd.to_datetime(df_do_preprocessingu_dla_nan['BuiltYear'], format='%Y', errors='coerce')

# Obsługa NaN w Description i Location przed CountVectorizer
df_do_preprocessingu_dla_nan['Description'] = df_do_preprocessingu_dla_nan['Description'].fillna('')
df_do_preprocessingu_dla_nan['Location'] = df_do_preprocessingu_dla_nan['Location'].fillna('')

# Przycięcie długości Description
df_do_preprocessingu_dla_nan['Description'] = df_do_preprocessingu_dla_nan['Description'].str.slice(0, 3000)

# Zastosowanie nauczonego CountVectorizer
print("Transformuję kolumnę 'Description' za pomocą CountVectorizer...")
X_bow_dla_nan = vectorizer.transform(df_do_preprocessingu_dla_nan["Description"])
df_bow_dla_nan = pd.DataFrame(X_bow_dla_nan.toarray(), columns=vectorizer.get_feature_names_out(), index=df_do_preprocessingu_dla_nan.index)

# Połącz cechy Bag-of-Words z resztą DataFrame
df_przetworzone_dla_nan = df_do_preprocessingu_dla_nan.drop(columns=['Description'])
df_przetworzone_dla_nan = pd.concat([df_przetworzone_dla_nan, df_bow_dla_nan], axis=1)
print(f"Dane po dodaniu cech BoW: {df_przetworzone_dla_nan.shape[1]} kolumn.")

# Konwersja typów kolumn numerycznych na string (jeśli były tak traktowane w setup)
string_conversion_cols = ['VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber', 'RegionNumber']
if 'StreetNumber' in df_przetworzone_dla_nan.columns:
    string_conversion_cols.append('StreetNumber')

print(f"Konwertuję kolumny na string: {string_conversion_cols}")
for col in string_conversion_cols:
    if col in df_przetworzone_dla_nan.columns:
        df_przetworzone_dla_nan[col] = df_przetworzone_dla_nan[col].astype(str)

# --- Identyfikacja wierszy z NaN i predykcja TYLKO dla nich ---
nan_indices = df_oryginal_do_wypelnienia_nan[df_oryginal_do_wypelnienia_nan['BuildingCondition'].isna()].index
print(f"Znaleziono {len(nan_indices)} wierszy z brakującym 'BuildingCondition' do wypełnienia.")

if len(nan_indices) > 0:
    # Wybierz te wiersze z przygotowanego df_przetworzone_dla_nan
    dane_do_predykcji_nan = df_przetworzone_dla_nan.loc[nan_indices].copy() # .copy() aby uniknąć SettingWithCopyWarning

    # WAŻNE: Usuń kolumnę 'BuildingCondition' z danych, na których robisz predykcję,
    # ponieważ `predict_model` zgłosi błąd, jeśli ją znajdzie z NaN-ami.
    if 'BuildingCondition' in dane_do_predykcji_nan.columns:
        print("Usuwam kolumnę 'BuildingCondition' z danych przekazywanych do predict_model (dla wierszy z NaN).")
        dane_do_predykcji_nan = dane_do_predykcji_nan.drop(columns=['BuildingCondition'])
    
    print("Wykonuję predykcje dla wierszy z brakującym 'BuildingCondition'...")
    
    # Użyj modelu, który został wybrany/sfinalizowany
    # (zakładamy, że model_do_uzycia_w_skrypcie_predykcji jest poprawnie zdefiniowany wcześniej)
    if 'model_do_uzycia_w_skrypcie_predykcji' not in locals() or model_do_uzycia_w_skrypcie_predykcji is None:
        # Próba wczytania, jeśli zmienna nie istnieje
        try:
            from pycaret.classification import load_model
            print("Zmienna 'model_do_uzycia_w_skrypcie_predykcji' nie znaleziona, próbuję wczytać 'model_LGBM_stan'...")
            model_do_uzycia_w_skrypcie_predykcji = load_model("model_LGBM_stan")
            print("Model wczytany pomyślnie.")
        except Exception as e:
            print(f"BŁĄD: Nie udało się wczytać modelu 'model_LGBM_stan': {e}")
            raise NameError("Nie zdefiniowano ani nie udało się wczytać modelu do predykcji.")


    predykcje_dla_nan_obj = transformed_exp.predict_model(
        estimator=model_do_uzycia_w_skrypcie_predykcji,
        data=dane_do_predykcji_nan
    )
    
    # 'prediction_label' to domyślna nazwa kolumny z predykcjami w PyCaret
    przewidziane_wartosci = predykcje_dla_nan_obj['prediction_label']
    
    # Wypełnij brakujące wartości w ORYGINALNYM DataFrame
    # Używamy .loc z oryginalnymi indeksami (nan_indices), aby upewnić się, że przypisujemy do właściwych miejsc
    df_oryginal_do_wypelnienia_nan.loc[nan_indices, 'BuildingCondition'] = przewidziane_wartosci.values

    print("\nZakończono wypełnianie NaN. Przykładowe wiersze, które miały NaN:")
    print(df_oryginal_do_wypelnienia_nan.loc[nan_indices, ['SaleId', 'Title', 'BuildingCondition']].head(10))
    print(f"Liczba NaN w 'BuildingCondition' PO wypełnieniu: {df_oryginal_do_wypelnienia_nan['BuildingCondition'].isna().sum()}")

else:
    print("Nie znaleziono wierszy z brakującym 'BuildingCondition' do wypełnienia.")

# Zapisz wynikowy DataFrame do nowego pliku CSV
output_filename_nan_filled = 'data_stan_nan_filled.csv'
df_oryginal_do_wypelnienia_nan.to_csv(output_filename_nan_filled, index=False, sep=',')
print(f"\nZapisano DataFrame z wypełnionymi NaN w 'BuildingCondition' do pliku: {output_filename_nan_filled}")

In [ ]:
df_nan = pd.read_csv('data_stan_nan_filled.csv', sep=',')

In [ ]:
df_nan.info()

In [ ]:
df_nan.head(10)

In [ ]:
# Nazwy plików
original_data_file = 'data.csv'
nan_filled_data_file = 'data_stan_nan_filled.csv' # Plik wygenerowany w poprzednim kroku
output_file_full = 'Data_state_out_full.csv'

print(f"Wczytuję oryginalne dane z: {original_data_file}")
df_original = pd.read_csv(original_data_file, sep=',')
print(f"Oryginalne dane wczytane. Kształt: {df_original.shape}")

print(f"Wczytuję dane z wypełnionymi NaN w BuildingCondition z: {nan_filled_data_file}")
df_nan_filled = pd.read_csv(nan_filled_data_file, sep=',')
print(f"Dane z wypełnionymi NaN wczytane. Kształt: {df_nan_filled.shape}")

# Sprawdzenie, czy oba DataFrame'y mają taką samą liczbę wierszy i najlepiej ten sam porządek
# (zakładamy, że tak jest, bo df_nan_filled powstał z df_original)
if df_original.shape[0] != df_nan_filled.shape[0]:
    print("OSTRZEŻENIE: Liczba wierszy w pliku oryginalnym i pliku z wypełnionymi NaN nie zgadza się!")
    print("Sprawdź, czy pliki są poprawne. Przerywam operację, aby uniknąć błędnego połączenia.")
    # Można by tu dodać bardziej zaawansowaną logikę łączenia po ID, jeśli kolejność nie jest gwarantowana,
    # ale na razie zakładamy, że kolejność jest zachowana.
else:
    # Tworzymy nową kolumnę 'Predict_State' w oryginalnym DataFrame
    # na podstawie (teraz już wypełnionej) kolumny 'BuildingCondition' z df_nan_filled.
    # Kolumna 'BuildingCondition' w df_nan_filled zawiera oryginalne wartości tam, gdzie były,
    # oraz przewidziane wartości tam, gdzie były NaN.
    
    print("Tworzę kolumnę 'Predict_State' w oryginalnym DataFrame...")
    df_original['Predict_State'] = df_nan_filled['BuildingCondition']

    # Wyświetlmy fragment, aby zobaczyć efekt
    print("\nPrzykładowe wiersze z oryginalną kolumną 'BuildingCondition' i nową 'Predict_State':")
    # Wybierzmy wiersze, gdzie oryginalnie było NaN, aby zobaczyć efekt wypełnienia w Predict_State
    original_nan_indices_for_display = df_original[df_original['BuildingCondition'].isna()].index[:5]
    if not original_nan_indices_for_display.empty:
        print("Przykłady, gdzie oryginalnie było NaN w BuildingCondition:")
        print(df_original.loc[original_nan_indices_for_display, ['SaleId', 'BuildingCondition', 'Predict_State']])
    
    # Przykłady, gdzie oryginalnie NIE było NaN
    original_not_nan_indices_for_display = df_original[df_original['BuildingCondition'].notna()].index[:5]
    if not original_not_nan_indices_for_display.empty:
        print("\nPrzykłady, gdzie oryginalnie BuildingCondition miało wartość:")
        print(df_original.loc[original_not_nan_indices_for_display, ['SaleId', 'BuildingCondition', 'Predict_State']])

    # Zapisz wynikowy DataFrame
    print(f"\nZapisuję połączony DataFrame do: {output_file_full}")
    df_original.to_csv(output_file_full, index=False, sep=',')
    print("Zapisano pomyślnie.")

    print(f"\nFinalny kształt pliku wyjściowego: {df_original.shape}")
    print("Pierwsze kilka wierszy pliku wyjściowego:")
    print(df_original.head())

In [ ]:
df_nan2 = pd.read_csv('Data_state_out_full.csv', sep=',')

In [ ]:
df_nan2

In [ ]:
display(df_nan2[['SaleId', 'Title', 'Area', 'Price', 'BuildingCondition', 'Predict_State']].sample(25))